# LatinBERT Gen
This approach will follow similarly to bert-babble as generated in the paper "Bert has a voice let it speak, BERT as an RMF" by X and Y at Z.

In [4]:
import numpy as np
import os, re
from Data import dataExp
%matplotlib inline
from matplotlib import pyplot as plt
import LatinBERT
from LatinBERT.gen_berts import LatinBERT
from LatinBERT.LatinTok import LatinTokenizer
from LatinBERT.predict_words import predict, infilling
from transformers import BertModel, BertForMaskedLM, BertPreTrainedModel
from tensor2tensor.data_generators import text_encoder
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=UserWarning)
    from cltk.tokenizers.lat.lat import LatinWordTokenizer as WordTokenizer
    from cltk.tokenizers.lat.lat import LatinPunktSentenceTokenizer as SentenceTokenizer
from cltk.embeddings.embeddings import Word2VecEmbeddings as W2VE
from sklearn import metrics
import pandas as pd
import torch
torch.cuda.empty_cache()

In [2]:
CI = dataExp.CorpusInterface(corpus_name="tokenized_corpus.pickle", shouldTokenize = True)

Found the existing corpus
abbofloracensis had 1 pieces of work with a total of 5403 characters of text
abelard had 1 pieces of work with a total of 13784 characters of text
acticussincerius had 1 pieces of work with a total of 340 characters of text
addison had 1 pieces of work with a total of 296 characters of text
adso had 1 pieces of work with a total of 2351 characters of text
aelredus had 1 pieces of work with a total of 20901 characters of text
agnes had 1 pieces of work with a total of 11811 characters of text
alanus had 1 pieces of work with a total of 34012 characters of text
albericodamarcellise had 1 pieces of work with a total of 27 characters of text
albertanus had 1 pieces of work with a total of 2869 characters of text
albertofaix had 1 pieces of work with a total of 14123 characters of text
alcuin had 1 pieces of work with a total of 487 characters of text
aleandrogerolamo had 1 pieces of work with a total of 654 characters of text
alfonsi had 1 pieces of work with a to

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizerPath = os.getcwd()+"/LatinBERT/latin.subword.encoder"
bertPath = os.getcwd()+"/LatinBERT/latin_bert"
encoder = text_encoder.SubwordTextEncoder(tokenizerPath)
tokenizer = LatinTokenizer(encoder)

model = BertForMaskedLM.from_pretrained(bertPath)
model.to(device)
model.eval()

Some weights of the model checkpoint at /notebooks/LatinBERT/latin_bert were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [5]:
tokenizer.reverseVocab

{0: '[PAD]',
 1: '[UNK]',
 2: '[CLS]',
 3: '[SEP]',
 4: '[MASK]',
 5: '<pad>_',
 6: '<EOS>_',
 7: ',_',
 8: '._',
 9: '_',
 10: 'et_',
 11: '-_',
 12: 'in_',
 13: ';_',
 14: 's_',
 15: 'est_',
 16: 'a_',
 17: ':_',
 18: 'ne_',
 19: 'i_',
 20: 'non_',
 21: 'm_',
 22: 'ad_',
 23: 'e_',
 24: 'o_',
 25: 'quod_',
 26: ')_',
 27: 'um_',
 28: '^_',
 29: 't_',
 30: 'de_',
 31: 'ut_',
 32: '(_',
 33: '&_',
 34: 'us_',
 35: 'que_',
 36: 'cum_',
 37: 'is_',
 38: 'qui_',
 39: 'sed_',
 40: 'si_',
 41: 'ex_',
 42: 'per_',
 43: 'quae_',
 44: 'apos_',
 45: '"_',
 46: '?_',
 47: 'hoc_',
 48: '1_',
 49: 'sunt_',
 50: 'esse_',
 51: 'c_',
 52: 'se_',
 53: 'autem_',
 54: 'enim_',
 55: 'quam_',
 56: 'quia_',
 57: '2_',
 58: '*_',
 59: 're_',
 60: ']_',
 61: 'vel_',
 62: 'es_',
 63: 'ab_',
 64: 'r_',
 65: 'etiam_',
 66: '—_',
 67: 'tur_',
 68: 'nt_',
 69: 'am_',
 70: 'it_',
 71: '3_',
 72: 'te_',
 73: '[_',
 74: 'sit_',
 75: 'p_',
 76: 'l_',
 77: 'ur_',
 78: '4_',
 79: '»_',
 80: 'bus_',
 81: 'em_',
 82: 'au

In [4]:


def tokenize_batch(batch):
    return [tokenizer.convert_tokens_to_ids(sent) for sent in batch]

def untokenize_batch(batch):
    results = []
    for sent in batch:
        tmp_sent = []
        for word in sent:
            predicted_word="%s%s" % (tokenizer.reverseVocab[word], "")
            predicted_word=re.sub("_$", "", predicted_word).lower()
            tmp_sent.append(predicted_word)
        results.append(tmp_sent)
    #return [tokenizer.convert_ids_to_tokens(sent) for sent in batch]
    return results

def detokenize(sent):
    """ Roughly detokenizes (mainly undoes wordpiece) """
    new_sent = []
    for i, tok in enumerate(sent):
        if tok.startswith("##"):
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[2:]
        else:
            new_sent.append(tok)
    return new_sent

CLS = '[CLS]'
SEP = '[SEP]'
MASK = '[MASK]'
mask_id = tokenizer.convert_tokens_to_ids([MASK])[0]
sep_id = tokenizer.convert_tokens_to_ids([SEP])[0]
cls_id = tokenizer.convert_tokens_to_ids([CLS])[0]

In [5]:
def generate_step(out, gen_idx, temperature=None, top_k=0, sample=False, return_list=True):
    """ Generate a word from from out[gen_idx]
    
    args:
        - out (torch.Tensor): tensor of logits of size batch_size x seq_len x vocab_size
        - gen_idx (int): location for which to generate for
        - top_k (int): if >0, only sample from the top k most probable words
        - sample (Bool): if True, sample from full distribution. Overridden by top_k 
    """

    
    logits = out["logits"][:, gen_idx]
    if temperature is not None:
        logits = logits / temperature
    if top_k > 0:
        kth_vals, kth_idx = logits.topk(top_k, dim=-1)
        dist = torch.distributions.categorical.Categorical(logits=kth_vals)
        idx = kth_idx.gather(dim=1, index=dist.sample().unsqueeze(-1)).squeeze(-1)
    elif sample:
        dist = torch.distributions.categorical.Categorical(logits=logits)
        idx = dist.sample().squeeze(-1)
    else:
        idx = torch.argmax(logits, dim=-1)
    return idx.tolist() if return_list else idx

In [6]:
# Generation modes as functions
import math
import time
from LatinBERT.predict_words import proc

def get_init_text(seed_text, max_len, batch_size = 1, rand_init=False):
    """ Get initial sentence by padding seed_text with either masks or random words to max_len """
    batch = [seed_text + [MASK] * max_len + [SEP] for _ in range(batch_size)]
    #if rand_init:
    #    for ii in range(max_len):
    #        init_idx[seed_len+ii] = np.random.randint(0, len(tokenizer.vocab))
    
    return tokenize_batch(batch)

def parallel_sequential_generation(seed_text, max_len=15, top_k=0, temperature=None, max_iter=300, burnin=200,
                                   cuda=False, print_every=10, verbose=True):
    """ Generate for one random position at a timestep
    
    args:
        - burnin: during burn-in period, sample from full distribution; afterwards take argmax
    """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, batch_size)
    
    for ii in range(max_iter):
        kk = np.random.randint(0, max_len)
        for jj in range(batch_size):
            batch[jj][seed_len+kk] = mask_id
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp)
        topk = top_k if (ii >= burnin) else 0
        idxs = generate_step(out, gen_idx=seed_len+kk, top_k=topk, temperature=temperature, sample=(ii < burnin))
        for jj in range(batch_size):
            batch[jj][seed_len+kk] = idxs[jj]
            
        if verbose and np.mod(ii+1, print_every) == 0:
            for_print = tokenizer.convert_ids_to_tokens(batch[0])
            for_print = for_print[:seed_len+kk+1] + ['(*)'] + for_print[seed_len+kk+1:]
            print("iter", ii+1, " ".join(for_print))
            
    return untokenize_batch(batch)

def parallel_generation(seed_text, max_len=15, top_k=0, temperature=None, max_iter=300, sample=True, 
                        cuda=False, print_every=10, verbose=True):
    """ Generate for all positions at a time step """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, batch_size)
    
    for ii in range(max_iter):
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp)
        for kk in range(max_len):
            idxs = generate_step(out, gen_idx=seed_len+kk, top_k=top_k, temperature=temperature, sample=sample)
            for jj in range(batch_size):
                batch[jj][seed_len+kk] = idxs[jj]
            
        if verbose and np.mod(ii, print_every) == 0:
            print("iter", ii+1, " ".join(tokenizer.convert_ids_to_tokens(batch[0])))
    
    return untokenize_batch(batch)
            
def sequential_generation(seed_text, batch_size=2, max_len=15, leed_out_len=15, 
                          top_k=0, temperature=None, sample=True, cuda=False):
    """ Generate one word at a time, in L->R order """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, batch_size)
    batch = batch.cuda() if cuda else batch
    
    for ii in range(max_len):
        inp = [sent[:seed_len+ii+leed_out_len]+[sep_id] for sent in batch]
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp)
        idxs = generate_step(out, gen_idx=seed_len+ii, top_k=top_k, temperature=temperature, sample=sample)
        for jj in range(batch_size):
            batch[jj][seed_len+ii] = idxs[jj]
        
        return untokenize_batch(batch)


def generate(n_samples, tokenizer, seed_text="[CLS]", batch_size=10, max_len=25, 
             sample=True, top_k=100, temperature=1.0, burnin=200, max_iter=500,
             cuda=False, print_every=1):
    # main generation function to call
    sentences = []
    n_batches = math.ceil(n_samples / batch_size)
    start_time = time.time()
    for batch_n in range(n_batches):
        batch = parallel_sequential_generation(seed_text, max_len=max_len, top_k=top_k,
                                               temperature=temperature, burnin=burnin, max_iter=max_iter, 
                                               cuda=cuda, verbose=False)
        
        #batch = sequential_generation(seed_text, batch_size=20, max_len=max_len, top_k=top_k, temperature=temperature, leed_out_len=leed_out_len, sample=sample)
        #batch = parallel_generation(seed_text, max_len=max_len, top_k=top_k, temperature=temperature, sample=sample, max_iter=max_iter)
        print(batch)
        if (batch_n + 1) % print_every == 0:
            print("Finished batch %d in %.3fs" % (batch_n + 1, time.time() - start_time))
            start_time = time.time()
        
        sentences += batch
    return sentences

In [7]:
# Utility functions

def printer(sent, should_detokenize=True):
    if should_detokenize:
        sent = detokenize(sent)[1:-1]
    print(" ".join(sent))
    
def read_sents(in_file, should_detokenize=False):
    sents = [sent.strip().split() for sent in open(in_file).readlines()]
    if should_detokenize:
        sents = [detokenize(sent) for sent in sents]
    return sents

def write_sents(out_file, sents, should_detokenize=False):
    with open(out_file, "w") as out_fh:
        for sent in sents:
            sent = detokenize(sent[1:-1]) if should_detokenize else sent
            out_fh.write("%s\n" % " ".join(sent))

In [8]:
n_samples = 1
batch_size = 10
max_len = 40
top_k = 100
temperature = 0.7

leed_out_len = 5 # max_len
burnin = 250
sample = True
max_iter = 500

# Choose the prefix context
seed_text = "[CLS]".split()

for temp in [1.0]:
    bert_sents = generate(n_samples, tokenizer, seed_text=seed_text, batch_size=batch_size, max_len=max_len,
                          sample=sample, top_k=top_k, temperature=temperature, burnin=burnin, max_iter=max_iter,
                          cuda=True)
    out_file = "Results/len%d-burnin%d-topk%d-temp4%.3f.txt" % (max_len, burnin, top_k, temp)
    write_sents(out_file, bert_sents, should_detokenize=True)

[['[cls]', 'init', '', '.', 'init', '', '.', 'praemi', 'tte', 'bd', 'm', '-', 'ne', 's', '.', 'congr', '', '.', ':', '(', '1', ')', 'aa', '.', 'de', 'matr', '', '.', '1', '.', 'et', '2', '.', 'de', 'matrimonio', 'toro', '', 'conco', 'mita', 'nte', '.', '[sep]'], ['[cls]', 'eme', '', 'eft', 'o', 'eme', '', 'efte', '', ':', 'eft', 'o', 'eme', '', 'efte', '', 'e', '€', '€', '€', '€', '€', '€', '€', '€', '€', '€', ':', 'eft', 'o', 'eme', '', '-', 'efte', '', 'e', '€', '€', '€', '€', '.', '[sep]'], ['[cls]', 'apo', 'phi', 'terat', 'orum', '.', 'apo', 'phi', 'terat', 'orum', '.', 'asc', '', 'aga', 'neus', '', '.', 'asc', '', 'aga', 'nus', '.', 'aga', 'neus', '', '.', 'aga', 'nus', '.', 'alc', 'ani', 'us', '.', 'aga', '', '-', 'ne', '.', 'aga', 'nus', '.', '[sep]'], ['[cls]', '16', 'pr', '.', 'modesti', 'nus', 'de', 'demi', '', '-', 'ne', '.', '1', 'uiui', 'tian', 'a', ']', 'uiui', 'tian', 'a', 'u', ',', 'ue', 'xta', 'f', '^', ',', 'ue', 'xta', '(', 'uiui', 'tian', 'a', 'f', ')', 'u', 'uiui',

In [5]:
predict(tokenizer, "Pedicabo ego vos et irrumabo Aureli pathice et cinaede Furi, qui me ex versiculss meis putastis quod sunt molliculi, parum pudicum. Nam castum esse decet pium poetam ipsum, versiculos nihil necesse est qui tum denique habent salem ac leporem Si sint molliculī ac parum pudici et quod pruriat incitāre possint ,", model)

'Pedicabo ego vos et irrumabo Aureli pathice et cinaede Furi, qui me ex versiculss meis putastis quod sunt molliculi, parum pudicum. Nam castum esse decet pium poetam ipsum, versiculos nihil necesse est qui tum denique habent salem ac leporem Si sint molliculī ac parum pudici et quod pruriat incitāre possint , ut'

In [28]:
context = "rufus"
for i in range(50):
    context = predict(tokenizer, context, model, sampleK=True)
print(context)

Rufus faciendi et formaliter dici posse sens, ut [ b + j [ bc / , aut ita uti aliquem aliquem ad alia similia : quo minus id efficere aliquis possit ; et ita facere aliquem alicujus ope . vel vel aliquid alicujus vel aliquod eorum vel ea qua ad illa


In [25]:
predict(tokenizer, "Arma", model, sampleK=True)

'Arma in'

In [9]:
infilling(tokenizer, "Arma virumque cano, Troiae qui primus ab oris Italiam, fato profugus, Laviniaque venit litora, multum ille et terris iactatus et alto vi superum saevae memorem", "ob iram; multa quoque et bello passus, dum conderet urbem,", model)

'Arma virumque cano, Troiae qui primus ab oris Italiam, fato profugus, Laviniaque venit litora, multum ille et terris iactatus et alto vi superum saevae memorem iunonis ob iram; multa quoque et bello passus, dum conderet urbem,'